##### Формирование и запись вспомогательной таблицы pre_contracts: "все договоры ИНН на день принятия решения"

Определим договоры по каждому ИНН + дата принятия решения в рамках 5 лет

Количество этих договоров будет фичей contract_cnt,
список договоров будет нужен для того, чтобы посчитать была ли у них просрочка и дата последнего оплаченного платежа

записываем эти договоры с датами принятия решения во вспомогательную таблицу UAKRPDB PRE_CONTRACTS

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/past_contracts.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

127968


In [4]:
name = pre + "data/interm/external_pkl/portfolio.pkl"
portfolio = pd.read_pickle(name, compression={'method':'zip'})

print(len(portfolio))
# portfolio.head(1)

268031


In [5]:
#  определяем все договоры, которые были начаты до даты принятия решения по сделке,
# но закончились не ранее -5 лет от даты принятия решения

portfolio['open_auxilary'] = portfolio['REHIRING_DT'].combine_first(portfolio['OPEN_DT'])

merge = pd.merge(target_data, portfolio, how='inner', left_on='ИНН контрагента', right_on='INN')

mask_open = (merge['Дата принятия решения по сделке']>merge['open_auxilary'])

# timedelta(days=2) - для тестирования, так как этих данных еще не было на prod, в рабочей версии делать не надо 
mask_leasing = (merge['Дата принятия решения по сделке']>merge['LEASE_DATE']+timedelta(days=1))

# без тех, что закончились более 5 лет назад
mask_closed = (merge['Дата принятия решения по сделке']<=merge['CLOSE_DT']+timedelta(days=1825)) # 365*5

last_contracts = merge[mask_open&mask_closed&mask_leasing].copy()

# last_contracts.head(1)

для будущих расчетов нам понадобятся связка Дата принятия решения по сделке + CONTRACT_KEY +CONTRACT_APP_KEY

то есть необходимо считать был ли просрочен каждый договор CONTRACT_KEY +CONTRACT_APP_KEY на дату принятия решения по сделке

In [6]:
columns = ['decision_date', 'CONTRACT_KEY', 'CONTRACT_NUM', 'CONTRACT_APP_KEY', 'INN']
pre_contracts = last_contracts[columns].drop_duplicates()
print(len(pre_contracts))

# pre_contracts.head(1)

207598


In [7]:
pre_contracts.dtypes

decision_date       datetime64[ns]
CONTRACT_KEY                object
CONTRACT_NUM                object
CONTRACT_APP_KEY            object
INN                         object
dtype: object

Запись во вспомогательную таблицу

In [8]:
cnxn_oracle = cx_Oracle.connect(obi_login, obi_pw, "10.0.2.45:1521/UAKRPDB", encoding="UTF-8") 
    
try:
    cursor = cnxn_oracle.cursor()
    # there is no drop if exists in Oracle
    sql_query = "select count(*) from user_tables where table_name = upper('Pre_contracts')"
    cursor.execute(sql_query)
    
    results = cursor.fetchall()
    
    for data in results:
        if (data[0]==1):
            sql_query = "DROP TABLE Pre_contracts"
            cursor.execute(sql_query)

    # create table 
    sql_query = """CREATE TABLE PRE_CONTRACTS
                    (
                        CONTRACT_KEY     NUMBER NOT NULL,
                        DECISION_DATE     DATE,
                        CONTRACT_NUM     VARCHAR2(1000 BYTE),
                        CONTRACT_APP_KEY NUMBER,
                        INN              VARCHAR2(100 CHAR)
                    )    
                """

    cursor.execute(sql_query)
    
    # запись блоком по 1000 строк
    rows_in_block = 0 
    insert_row = ""
    for row, val in tqdm(pre_contracts.iterrows(), total = len(pre_contracts)):

        insert_row += """INTO PRE_CONTRACTS (CONTRACT_KEY, DECISION_DATE, CONTRACT_NUM , CONTRACT_APP_KEY, INN) VALUES 
                    ({}, TO_DATE(\'{}\', \'yyyy-mm-dd hh24:mi:ss\'),  \'{}\',  {}, \'{}\') """.format(
                        val['CONTRACT_KEY'], val['decision_date'], val['CONTRACT_NUM'], val['CONTRACT_APP_KEY'], val['INN'])
        
        rows_in_block += 1

        if rows_in_block == 1000:
            query = """INSERT ALL """ + insert_row + "SELECT * FROM dual"
            rows_in_block = 0
            insert_row = ""
            cursor.execute(query)

    if rows_in_block>0:
        query = """INSERT ALL """ + insert_row + "SELECT * FROM dual"
        rows_in_block = 0
        insert_row = ""
        cursor.execute(query)
    
    cnxn_oracle.commit()
    
finally:
    
    if cursor:
        cursor.close()
    if cnxn_oracle:
        cnxn_oracle.close()

100%|██████████| 207598/207598 [05:14<00:00, 660.74it/s]


In [9]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 